In [1]:
import os
import glob
import rasterio
import numpy as np
import geopandas as gpd
import folium
from rasterio.features import shapes
from shapely.geometry import shape, MultiPolygon

In [30]:

# Fonction pour extraire les polygones par classe
def extract_polygons_from_tif(tif_path):
    with rasterio.open(tif_path) as src:
        image = src.read(1)  # Lire la première bande (classes)
        transform = src.transform  # Récupérer la transformation géospatiale
        
        # Obtenir les polygones des classes
        for shape_dict, value in shapes(image, transform=transform):
            if value > 0:  # On ignore la classe 0 si elle représente le fond
                features.append({"geometry": shape(shape_dict), "class": int(value)})

# Appliquer la fonction à chaque fichier
for tif_file in tif_files:
    extract_polygons_from_tif(tif_file)

# Convertir en GeoDataFrame
gdf = gpd.GeoDataFrame(features, crs="EPSG:3035")  # Lambert-93

# Sauvegarder en shapefile (facultatif)
#gdf.to_file(f"{label_dir}/concatenated_labels.shp")

gdf = gdf.to_crs(4326)
print("Conversion terminée, affichage sur Folium...")

# --- Étape 5 : Afficher sur Folium ---
# Déterminer le centre de la carte

largest_polygon = gdf.geometry.area.idxmax()  # Index du plus grand polygone
center_x, center_y = gdf.geometry.centroid.iloc[largest_polygon].coords[0]

m = folium.Map(location=[center_y, center_x], zoom_start=15)


# Définir les couleurs adaptées aux classes
class_colors = {
    1: "#8B0000",  # Rouge foncé - Sealed
    2: "#006400",  # Vert foncé - Conifères
    3: "#228B22",  # Vert forêt - Feuillus caducs
    4: "#32CD32",  # Vert clair - Feuillus persistants
    5: "#8FBC8F",  # Vert gris - Arbustes
    6: "#ADFF2F",  # Vert jaune - Herbacées permanentes
    7: "#FFD700",  # Jaune - Herbacées saisonnières
    8: "#D2B48C",  # Beige - Lichens/mousses
    9: "#A9A9A9",  # Gris foncé - Roches, désert
    10: "#1E90FF",  # Bleu vif - Eau
    11: "#FFFFFF",  # Blanc - Neige et glace
    254: "#000000",  # Noir - Hors zone
    255: "#808080"  # Gris - No data
}
gdf_filtered = gdf[gdf["class"] == 1]

# Ajouter chaque classe sur la carte Folium
for _, row in gdf_filtered.iterrows():
    folium.GeoJson(
        row.geometry,
        name=f"Class {row['class']}",
        style_function=lambda feature, color=class_colors.get(row["class"], "black"): {
            "fillColor": color,
            "color": color,
            "weight": 1,
            "fillOpacity": 0.5,
        },
    ).add_to(m)


Conversion terminée, affichage sur Folium...


/tmp/ipykernel_99646/520818002.py:28: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  largest_polygon = gdf.geometry.area.idxmax()  # Index du plus grand polygone
/tmp/ipykernel_99646/520818002.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_x, center_y = gdf.geometry.centroid.iloc[largest_polygon].coords[0]


## polygone de différence !

In [43]:

# --- Étape 1 : Définition des fichiers pour 2018 et 2021 ---
NUTS3 = "BE100"
label_dir = f"../../data-preprocessed/labels/CLCplus-Backbone/SENTINEL2/{NUTS3}"

# Lister les fichiers .tif pour 2018 et 2021
tif_2018_files = sorted(glob.glob(os.path.join(label_dir, "2018/250/*.tif")))
tif_2021_files = sorted(glob.glob(os.path.join(label_dir, "2021/250/*.tif")))

# Vérification que les listes de fichiers sont cohérentes
assert len(tif_2018_files) == len(tif_2021_files), "Nombre de fichiers différent entre 2018 et 2021 !"

# --- Étape 2 : Fonction pour binariser un raster (Classe 1 → 1, autres → 0) ---
def load_and_binarize_raster(path):
    """Charge un raster et binarise : classe 1 → 1, autres → 0"""
    with rasterio.open(path) as src:
        image = src.read(1)  # Lire la première bande
        transform = src.transform  # Transformation géospatiale
    binary_image = np.where(image == 1, 1, 0).astype(np.uint8)
    return binary_image, transform

# --- Étape 3 : Calculer la différence des rasters et extraire les polygones ---
features = []

for tif_2018, tif_2021 in zip(tif_2018_files, tif_2021_files):
    # Charger les rasters binarisés
    binary_2018, transform = load_and_binarize_raster(tif_2018)
    binary_2021, _ = load_and_binarize_raster(tif_2021)

    # Matrice de changement des classes
    change_matrix = binary_2018 * 2 + binary_2021  # (00 → 0, 01 → 1, 10 → 2, 11 → 3)

    # Extraction des polygones uniquement pour les zones ayant changé (≠ 0)
    for shape_dict, value in shapes(change_matrix, transform=transform):
        if value > 0:  # Exclure les zones sans changement
            features.append({"geometry": shape(shape_dict), "class": int(value)})

# --- Étape 4 : Convertir en GeoDataFrame ---
SRC_CRS = "EPSG:3035"  # Lambert-93
TARGET_CRS = "EPSG:4326"  # WGS 84 (GPS)

gdf = gpd.GeoDataFrame(features, crs=SRC_CRS).to_crs(TARGET_CRS)

# Sauvegarde en shapefile (optionnel)
gdf.to_file(f"{label_dir}/change_polygons.shp")

print("GeoDataFrame des changements créé avec succès !")
gdf = gdf[gdf["class"] != 0]


GeoDataFrame des changements créé avec succès !


In [56]:


# --- Étape 4 : Création d'une carte interactive Folium ---
# Définir les couleurs pour les 4 transitions
change_colors = {
    0: "#FFFFFF",  # Pas de changement (blanc)
    1: "#FF0000",  # Apparition de la classe 1 (rouge)
    2: "#0000FF",  # Disparition de la classe 1 (bleu)
    3: "#FACFCB",  # Classe 1 persistante (orange)
}

# Trouver le centroïde pour centrer la carte
largest_polygon_idx = gdf.geometry.area.idxmax()
center_x, center_y = gdf.geometry.centroid.iloc[largest_polygon_idx].coords[0]

# Créer la carte centrée sur ce point
m = folium.Map(location=[center_y, center_x], zoom_start=10)

# Ajouter chaque classe sur la carte
for _, row in gdf.iterrows():
    folium.GeoJson(
        row.geometry,
        name=f"Change {row['class']}",
        style_function=lambda feature, color=change_colors.get(row["class"], "#000000"): {
            "fillColor": color,
            "color": color,
            "weight": 1,
            "fillOpacity": 0.5,
        },
    ).add_to(m)



/tmp/ipykernel_99646/1020850332.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  largest_polygon_idx = gdf.geometry.area.idxmax()
/tmp/ipykernel_99646/1020850332.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_x, center_y = gdf.geometry.centroid.iloc[largest_polygon_idx].coords[0]


In [ ]:
m